# Metaclasses

**Chapter 4 - Learning Python, 5th Edition**

Metaclasses are the classes behind classes. In Python, classes themselves are objects—instances of their metaclass. By default, that metaclass is `type`. Understanding metaclasses unlocks powerful patterns for controlling class creation, enforcing constraints, and building frameworks.

## Section 1: Classes Are Objects

In [ ]:
# Classes are objects: type() reveals a class's type is `type`
class Dog:
    """A simple class."""
    pass

# An instance's type is its class
dog: Dog = Dog()
print(f"type(dog): {type(dog)}")
print(f"type(dog) is Dog: {type(dog) is Dog}")

# A class's type is `type` -- the default metaclass
print(f"\ntype(Dog): {type(Dog)}")
print(f"type(Dog) is type: {type(Dog) is type}")

# Even built-in types are instances of `type`
print(f"\ntype(int): {type(int)}")
print(f"type(str): {type(str)}")
print(f"type(list): {type(list)}")

# And `type` is its own metaclass
print(f"\ntype(type): {type(type)}")
print(f"type(type) is type: {type(type) is type}")

# Everything is an object in Python
print(f"\nisinstance(Dog, object): {isinstance(Dog, object)}")
print(f"isinstance(type, object): {isinstance(type, object)}")
print(f"isinstance(42, object): {isinstance(42, object)}")

## Section 2: `type()` as a Class Factory

In [ ]:
# type(name, bases, dict) creates classes dynamically

# Traditional class statement
class Dog:
    species: str = "Canis familiaris"

    def __init__(self, name: str) -> None:
        self.name = name

    def bark(self) -> str:
        return f"{self.name} says Woof!"

# Equivalent using three-argument type()
def dog_init(self: 'DynamicDog', name: str) -> None:
    self.name = name

def dog_bark(self: 'DynamicDog') -> str:
    return f"{self.name} says Woof!"

DynamicDog = type(
    "DynamicDog",                              # class name
    (object,),                                  # base classes (tuple)
    {                                           # class body namespace
        "species": "Canis familiaris",
        "__init__": dog_init,
        "bark": dog_bark,
    },
)

# Both produce the same behavior
dog1: Dog = Dog("Buddy")
dog2: DynamicDog = DynamicDog("Max")

print(f"dog1.bark(): {dog1.bark()}")
print(f"dog2.bark(): {dog2.bark()}")

print(f"\ntype(Dog): {type(Dog)}")
print(f"type(DynamicDog): {type(DynamicDog)}")

# Dynamic class creation is useful for factory patterns
print(f"\nDynamicDog.__name__: {DynamicDog.__name__}")
print(f"DynamicDog.species: {DynamicDog.species}")

## Section 3: `__new__` vs `__init__` in Classes

In [ ]:
# __new__ creates the instance; __init__ initializes it
class Traced:
    """Demonstrates the two-phase object creation process."""

    def __new__(cls, value: int) -> 'Traced':
        print(f"__new__ called: creating instance of {cls.__name__}")
        instance: Traced = super().__new__(cls)
        return instance

    def __init__(self, value: int) -> None:
        print(f"__init__ called: initializing with value={value}")
        self.value: int = value

print("Creating Traced(42):")
obj: Traced = Traced(42)
print(f"obj.value: {obj.value}")

print("\n" + "="*50)

# Practical use: customizing immutable types
# Immutable types (str, int, tuple) must be customized in __new__
# because by __init__ time, the value is already frozen
class UpperStr(str):
    """A string subclass that is always uppercase."""

    def __new__(cls, value: str) -> 'UpperStr':
        # Must modify in __new__ because str is immutable
        instance: UpperStr = super().__new__(cls, value.upper())
        return instance

greeting: UpperStr = UpperStr("hello world")
print(f"\nUpperStr('hello world'): {greeting!r}")
print(f"type(greeting): {type(greeting)}")
print(f"isinstance(greeting, str): {isinstance(greeting, str)}")

# ClampedInt: an int subclass that clamps to a range
class ClampedInt(int):
    """An int that clamps its value between 0 and 255."""

    def __new__(cls, value: int) -> 'ClampedInt':
        clamped: int = max(0, min(255, value))
        return super().__new__(cls, clamped)

print(f"\nClampedInt(300): {ClampedInt(300)}")
print(f"ClampedInt(-10): {ClampedInt(-10)}")
print(f"ClampedInt(128): {ClampedInt(128)}")

## Section 4: Writing a Basic Metaclass

In [ ]:
# A metaclass inherits from `type` and controls class creation
class VerboseMeta(type):
    """A metaclass that logs class creation."""

    def __new__(
        mcs,
        name: str,
        bases: tuple[type, ...],
        namespace: dict[str, object],
    ) -> 'VerboseMeta':
        print(f"VerboseMeta.__new__ creating class: {name!r}")
        print(f"  bases: {bases}")
        print(f"  namespace keys: {list(namespace.keys())}")
        cls = super().__new__(mcs, name, bases, namespace)
        return cls

    def __init__(
        cls,
        name: str,
        bases: tuple[type, ...],
        namespace: dict[str, object],
    ) -> None:
        print(f"VerboseMeta.__init__ finalizing class: {name!r}")
        super().__init__(name, bases, namespace)

# Using the metaclass keyword
class Animal(metaclass=VerboseMeta):
    sound: str = "..."

    def speak(self) -> str:
        return f"{type(self).__name__} says {self.sound}"

print(f"\ntype(Animal): {type(Animal)}")
print(f"Animal.sound: {Animal.sound}")

print()

# Subclasses inherit the metaclass
class Cat(Animal):
    sound: str = "Meow"

cat: Cat = Cat()
print(f"\ncat.speak(): {cat.speak()}")
print(f"type(Cat): {type(Cat)}")

## Section 5: Singleton Metaclass

In [ ]:
# Singleton: ensure only one instance per class
class SingletonMeta(type):
    """Metaclass that ensures a class has at most one instance."""

    _instances: dict[type, object] = {}

    def __call__(cls, *args: object, **kwargs: object) -> object:
        if cls not in cls._instances:
            # Create the single instance via normal type.__call__
            instance: object = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

class DatabaseConnection(metaclass=SingletonMeta):
    """Only one connection instance should exist."""

    def __init__(self, host: str = "localhost", port: int = 5432) -> None:
        self.host: str = host
        self.port: int = port

    def __repr__(self) -> str:
        return f"DatabaseConnection(host={self.host!r}, port={self.port})"

# First call creates the instance
db1: DatabaseConnection = DatabaseConnection("prod-server", 5432)
print(f"db1: {db1}")
print(f"id(db1): {id(db1)}")

# Second call returns the same instance (args are ignored)
db2: DatabaseConnection = DatabaseConnection("other-server", 3306)
print(f"\ndb2: {db2}")
print(f"id(db2): {id(db2)}")

print(f"\ndb1 is db2: {db1 is db2}")

# Each class using SingletonMeta gets its own singleton
class CacheManager(metaclass=SingletonMeta):
    def __init__(self, max_size: int = 100) -> None:
        self.max_size: int = max_size

cache: CacheManager = CacheManager(500)
print(f"\ncache is db1: {cache is db1}")
print("Note: For thread safety, wrap instance creation with threading.Lock")

## Section 6: Registry Metaclass

In [ ]:
# Auto-register all subclasses in a registry dict
class RegistryMeta(type):
    """Metaclass that auto-registers subclasses by name."""

    def __init__(
        cls,
        name: str,
        bases: tuple[type, ...],
        namespace: dict[str, object],
    ) -> None:
        super().__init__(name, bases, namespace)
        # Initialize the registry on the base class only
        if not hasattr(cls, '_registry'):
            cls._registry: dict[str, type] = {}
        else:
            # Subclass: register it
            cls._registry[name] = cls

# Plugin system base class
class Serializer(metaclass=RegistryMeta):
    """Base class for serializers. Subclasses are auto-registered."""

    def serialize(self, data: dict[str, object]) -> str:
        raise NotImplementedError

    @classmethod
    def get_serializer(cls, name: str) -> 'Serializer':
        """Factory: look up a serializer by name."""
        if name not in cls._registry:
            available: str = ", ".join(cls._registry.keys())
            raise ValueError(f"Unknown serializer {name!r}. Available: {available}")
        return cls._registry[name]()

# Define plugins -- they register themselves automatically
class JSONSerializer(Serializer):
    def serialize(self, data: dict[str, object]) -> str:
        import json
        return json.dumps(data)

class CSVSerializer(Serializer):
    def serialize(self, data: dict[str, object]) -> str:
        header: str = ",".join(data.keys())
        values: str = ",".join(str(v) for v in data.values())
        return f"{header}\n{values}"

class XMLSerializer(Serializer):
    def serialize(self, data: dict[str, object]) -> str:
        items: str = "".join(f"<{k}>{v}</{k}>" for k, v in data.items())
        return f"<root>{items}</root>"

# Discover available implementations
print(f"Registered serializers: {list(Serializer._registry.keys())}")

# Use the factory to get a serializer by name
sample_data: dict[str, object] = {"name": "Alice", "age": 30, "active": True}

for fmt in ["JSONSerializer", "CSVSerializer", "XMLSerializer"]:
    serializer: Serializer = Serializer.get_serializer(fmt)
    result: str = serializer.serialize(sample_data)
    print(f"\n{fmt}:")
    print(f"  {result}")

## Section 7: Validation Metaclass

In [ ]:
# Enforce class structure at definition time
class InterfaceMeta(type):
    """Metaclass that enforces required methods and naming conventions."""

    required_methods: tuple[str, ...] = ()

    def __new__(
        mcs,
        name: str,
        bases: tuple[type, ...],
        namespace: dict[str, object],
    ) -> 'InterfaceMeta':
        cls = super().__new__(mcs, name, bases, namespace)

        # Skip validation for the base class itself
        if bases:
            # Check required methods
            for method_name in mcs.required_methods:
                if method_name not in namespace:
                    raise TypeError(
                        f"Class {name!r} must implement {method_name!r}"
                    )

            # Enforce naming convention: class names must be CamelCase
            if not name[0].isupper():
                raise TypeError(
                    f"Class name {name!r} must start with uppercase letter"
                )

            # Enforce __slots__ usage
            if "__slots__" not in namespace:
                raise TypeError(
                    f"Class {name!r} must define __slots__ for memory efficiency"
                )

        return cls

# Create a specific interface metaclass for models
class ModelMeta(InterfaceMeta):
    required_methods: tuple[str, ...] = ("validate", "to_dict")

class BaseModel(metaclass=ModelMeta):
    """Base class for validated models."""
    pass

# Valid model: implements all requirements
class UserModel(BaseModel):
    __slots__ = ("name", "email")

    def __init__(self, name: str, email: str) -> None:
        self.name = name
        self.email = email

    def validate(self) -> bool:
        return "@" in self.email and len(self.name) > 0

    def to_dict(self) -> dict[str, str]:
        return {"name": self.name, "email": self.email}

user: UserModel = UserModel("Alice", "alice@example.com")
print(f"user.validate(): {user.validate()}")
print(f"user.to_dict(): {user.to_dict()}")

# Invalid model: missing required method
print("\nAttempting to define a class without required methods...")
try:
    class BadModel(BaseModel):
        __slots__ = ()

        def validate(self) -> bool:
            return True
        # Missing to_dict!
except TypeError as e:
    print(f"TypeError: {e}")

# Invalid model: missing __slots__
print("\nAttempting to define a class without __slots__...")
try:
    class NoSlotsModel(BaseModel):
        def validate(self) -> bool:
            return True

        def to_dict(self) -> dict[str, str]:
            return {}
except TypeError as e:
    print(f"TypeError: {e}")

## Section 8: `__init_subclass__` (Modern Alternative)

In [ ]:
# PEP 487: __init_subclass__ -- simpler than metaclasses for most use cases
# Available since Python 3.6

# Example 1: Auto-registration without a metaclass
class Plugin:
    """Base class that auto-registers subclasses using __init_subclass__."""

    _plugins: dict[str, type] = {}

    def __init_subclass__(cls, plugin_name: str = "", **kwargs: object) -> None:
        """Called automatically when a subclass is defined."""
        super().__init_subclass__(**kwargs)
        name: str = plugin_name or cls.__name__
        Plugin._plugins[name] = cls
        print(f"Registered plugin: {name!r}")

# Subclasses register automatically -- no metaclass needed
class AuthPlugin(Plugin, plugin_name="auth"):
    def execute(self) -> str:
        return "Authenticating..."

class LoggingPlugin(Plugin, plugin_name="logging"):
    def execute(self) -> str:
        return "Logging..."

class CachePlugin(Plugin):  # Uses class name as default
    def execute(self) -> str:
        return "Caching..."

print(f"\nRegistered plugins: {list(Plugin._plugins.keys())}")

print("\n" + "="*50)

# Example 2: Validation via __init_subclass__
class Validated:
    """Base class that enforces required methods on subclasses."""

    required_methods: tuple[str, ...] = ()

    def __init_subclass__(cls, **kwargs: object) -> None:
        super().__init_subclass__(**kwargs)
        for method_name in cls.required_methods:
            if not any(method_name in B.__dict__ for B in cls.__mro__[:-1]):
                raise TypeError(
                    f"{cls.__name__!r} must implement {method_name!r}"
                )

class Renderable(Validated):
    required_methods: tuple[str, ...] = ("render",)

class HTMLRenderer(Renderable):
    def render(self) -> str:
        return "<html>...</html>"

renderer: HTMLRenderer = HTMLRenderer()
print(f"\nrenderer.render(): {renderer.render()}")

# This would fail:
print("\nAttempting to define a class without 'render'...")
try:
    class BadRenderer(Renderable):
        pass
except TypeError as e:
    print(f"TypeError: {e}")

print("\n" + "="*50)
print("\nWhen to use which:")
print("  __init_subclass__: Registration, validation, simple hooks")
print("  Metaclass:         Full namespace control, __prepare__, complex frameworks")
print("  Class decorator:   Post-hoc modification, no inheritance needed")

## Summary

### Key Concepts

1. **Classes are objects** -- every class is an instance of `type` (the default metaclass)
2. **`type(name, bases, dict)`** -- the three-argument form creates classes dynamically at runtime
3. **`__new__` vs `__init__`** -- `__new__` creates the instance, `__init__` initializes it; use `__new__` for immutable type customization
4. **Metaclasses** inherit from `type` and override `__new__`/`__init__` to hook into class creation
5. **Singleton metaclass** -- override `__call__` to return the same instance on every instantiation
6. **Registry metaclass** -- auto-register subclasses for plugin/factory patterns
7. **Validation metaclass** -- enforce required methods, naming conventions, and `__slots__` at definition time
8. **`__init_subclass__` (PEP 487)** -- the modern, simpler alternative for most metaclass use cases

### When to Use What

| Approach | Use Case |
|---|---|
| `__init_subclass__` | Registration, simple validation, parameter passing |
| Metaclass | Full namespace control, `__prepare__`, complex frameworks |
| Class decorator | Post-hoc modification without inheritance |

### Rule of Thumb

Prefer `__init_subclass__` unless you need to control the class namespace before it is populated (via `__prepare__`) or need behavior that `__init_subclass__` cannot express.